# 식신 사이트 크롤링

In [2]:
from bs4 import BeautifulSoup 
import pandas as pd 
import requests
from urllib.parse import quote

In [3]:
quote('대구시청')

'%EB%8C%80%EA%B5%AC%EC%8B%9C%EC%B2%AD'

In [4]:
place = '산격동'
base_url = 'https://www.siksinhot.com'
sub_url = '/search?keywords='
url = base_url + sub_url + quote(place)
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [6]:
lis = soup.select_one('.listTy1').find_all('li')
len(lis)

60

In [9]:
a_list = soup.select_one('.listTy1').find('ul').find_all('a')
len(a_list)

12

In [10]:
a_tag = a_list[0]
rest_url = a_tag['href']
score = a_tag.select_one('.score').get_text()
store = a_tag.select_one('.store').get_text()
menu = a_tag.find('p').get_text()
rest_url, score, store, menu

('/P/268646', '4.4', '반짝반짝빛나는', '단호박퓨레 타르트, 밀크티')

In [11]:
url_list, score_list, store_list, menu_list = [], [], [], []
for a_tag in a_list:
    rest_url = a_tag['href']
    score = a_tag.select_one('.score').get_text()
    store = a_tag.select_one('.store').get_text()
    menu = a_tag.find('p').get_text()

    url_list.append(rest_url)
    score_list.append(score)
    store_list.append(store)
    menu_list.append(menu)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [13]:
a_tag = a_list[6]
rest_url = a_tag['href']
score = a_tag.select_one('.score2').get_text()
store = a_tag.select_one('.store').get_text()
menu = a_tag.find('p').get_text()
rest_url, score, store, menu

('/P/269346', '평가중', '봉봉해물탕', '해물탕\n, 해물찜')

In [15]:
url_list, score_list, store_list, menu_list = [], [], [], []
for a_tag in a_list:
    rest_url = a_tag['href']
    '''
    if a_tag.select_one('.score'):
        score = a_tag.select_one('.score').get_text()
    else:
        score = a_tag.select_one('.score2').get_text()
    '''
    score = a_tag.select_one('.score').get_text() if a_tag.select_one('.score') \
            else a_tag.select_one('.score2').get_text()
    store = a_tag.select_one('.store').get_text()
    menu = a_tag.find('p').get_text()

    url_list.append(rest_url)
    score_list.append(score)
    store_list.append(store)
    menu_list.append(menu)

In [16]:
df = pd.DataFrame({
    'url': url_list, '평점':score_list, '상호':store_list, '메뉴':menu_list
})
df

,url,평점,상호,메뉴
0,/P/268646,4.4,반짝반짝빛나는,"단호박퓨레 타르트, 밀크티"
1,/P/268652,3.5,인도방랑기,"탄두리치킨, 커리, 세트메뉴"
2,/P/268658,4.8,ZZIM,"한마리, 가든샐러드"
3,/P/355593,3.7,홍대함바그,"매콤함바그, 달콤함바그, 매콤갈비, 달콤갈비"
4,/P/268665,3.0,밥하는남자,"상하이누룽지, 통낙지라이스, 돈카츠샐러드"
5,/P/268649,3.0,포베트21,"양지 쇠고기 쌀국수, 세트(월남쌈+양지or해물쌀국수2개), 반세오, 월남쌈, 카오팟..."
6,/P/269346,평가중,봉봉해물탕,"해물탕\n, 해물찜"
7,/P/361407,4.3,72번가,"준벅, 카우보이정식, 코스모 폴리탄, 프렌치 키스"
8,/P/455234,4.3,META PIZZA,"미트디럭스피자, 쉬림프토파토 파스타, 로제, 프리마베라 피자, 쉬림프크림 파스타"
9,/P/1064049,평가중,모모야스시 참치,"모듬 스시 11P, 특스시 13P, 모모야스시 11P, 참치메뉴, 광어추가회"


### 사진을 클릭하여 레스토랑 페이지로 들어가서 크롤링

In [17]:
df.index

RangeIndex(start=0, stop=12, step=1)

In [18]:
rest_url = df.url[0]
url = base_url + rest_url
req = requests.get(url)
rest_soup = BeautifulSoup(req.text, 'html.parser')

In [20]:
tel = rest_soup.select_one('.p_tel').find('p').get_text()
tel

'053-954-8111'

In [21]:
addr = rest_soup.select_one('.txt_adr').get_text()
addr

'대구광역시 북구 대학로23길 12-1'

In [22]:
tel_list, addr_list = [], []
for i in df.index:
    rest_url = df.url[i]
    url = base_url + rest_url
    req = requests.get(url)
    rest_soup = BeautifulSoup(req.text, 'html.parser')

    tel = rest_soup.select_one('.p_tel').find('p').get_text()
    addr = rest_soup.select_one('.txt_adr').get_text()

    tel_list.append(tel)
    addr_list.append(addr)

In [23]:
df['전화번호'] = tel_list
df['주소'] = addr_list
df

,url,평점,상호,메뉴,전화번호,주소
0,/P/268646,4.4,반짝반짝빛나는,"단호박퓨레 타르트, 밀크티",053-954-8111,대구광역시 북구 대학로23길 12-1
1,/P/268652,3.5,인도방랑기,"탄두리치킨, 커리, 세트메뉴",053-956-9940,대구광역시 북구 대학로 81
2,/P/268658,4.8,ZZIM,"한마리, 가든샐러드",053-941-7272,대구 북구 대학로 61
3,/P/355593,3.7,홍대함바그,"매콤함바그, 달콤함바그, 매콤갈비, 달콤갈비",053-269-5008,대구광역시 북구 대학로23길 19-6
4,/P/268665,3.0,밥하는남자,"상하이누룽지, 통낙지라이스, 돈카츠샐러드",070-7584-1078,대구광역시 북구 대학로23길 25-2
5,/P/268649,3.0,포베트21,"양지 쇠고기 쌀국수, 세트(월남쌈+양지or해물쌀국수2개), 반세오, 월남쌈, 카오팟...",053-939-3688,
6,/P/269346,평가중,봉봉해물탕,"해물탕\n, 해물찜",053-383-8000,대구 북구 유통단지로7길 21
7,/P/361407,4.3,72번가,"준벅, 카우보이정식, 코스모 폴리탄, 프렌치 키스",053-939-0817,
8,/P/455234,4.3,META PIZZA,"미트디럭스피자, 쉬림프토파토 파스타, 로제, 프리마베라 피자, 쉬림프크림 파스타",053-939-6767,대구광역시 북구 대학로 103 지하1층
9,/P/1064049,평가중,모모야스시 참치,"모듬 스시 11P, 특스시 13P, 모모야스시 11P, 참치메뉴, 광어추가회",053-381-8999,대구광역시 북구 대불로 37


In [24]:
addr_div = rest_soup.select_one('.p_tel').next_sibling()
addr_div

[<h4 data-reactid="161">주소</h4>,
 <p data-reactid="162"><a class="txt_adr" data-reactid="163" href="#layer_store_big_map">대구 북구 대학로23길 7</a><span data-reactid="164"><!-- react-text: 165 -->(지번) <!-- /react-text --><!-- react-text: 166 -->대구광역시 북구 산격동 1321-5<!-- /react-text --></span><a class="btnTy1 btn_google_map" data-reactid="167" href="http://maps.google.com/maps?q=35.894658,128.611049" target="_blank">구글 지도로 보기</a></p>,
 <a class="txt_adr" data-reactid="163" href="#layer_store_big_map">대구 북구 대학로23길 7</a>,
 <span data-reactid="164"><!-- react-text: 165 -->(지번) <!-- /react-text --><!-- react-text: 166 -->대구광역시 북구 산격동 1321-5<!-- /react-text --></span>,
 <a class="btnTy1 btn_google_map" data-reactid="167" href="http://maps.google.com/maps?q=35.894658,128.611049" target="_blank">구글 지도로 보기</a>]

In [27]:
tel_list, addr_list = [], []
for i in df.index:
    rest_url = df.url[i]
    url = base_url + rest_url
    req = requests.get(url)
    rest_soup = BeautifulSoup(req.text, 'html.parser')

    tel_div = rest_soup.select_one('.p_tel')
    tel = tel_div.find('p').get_text()
    '''
    addr = rest_soup.select_one('.txt_adr').get_text()
    if not addr:
        addr_div = tel_div.next_sibling()
        addr = addr_div[1].find('span').get_text()
    '''
    addr_div = tel_div.next_sibling()
    addr = addr_div[1].select_one('.txt_adr').get_text() if addr_div[1].select_one('.txt_adr').get_text() \
           else addr_div[1].find('span').get_text()

    tel_list.append(tel)
    addr_list.append(addr)

In [28]:
del df['전화번호']
del df['주소']
del df['url']
df['전화번호'] = tel_list
df['주소'] = addr_list
df

,평점,상호,메뉴,전화번호,주소
0,4.4,반짝반짝빛나는,"단호박퓨레 타르트, 밀크티",053-954-8111,대구광역시 북구 대학로23길 12-1
1,3.5,인도방랑기,"탄두리치킨, 커리, 세트메뉴",053-956-9940,대구광역시 북구 대학로 81
2,4.8,ZZIM,"한마리, 가든샐러드",053-941-7272,대구 북구 대학로 61
3,3.7,홍대함바그,"매콤함바그, 달콤함바그, 매콤갈비, 달콤갈비",053-269-5008,대구광역시 북구 대학로23길 19-6
4,3.0,밥하는남자,"상하이누룽지, 통낙지라이스, 돈카츠샐러드",070-7584-1078,대구광역시 북구 대학로23길 25-2
5,3.0,포베트21,"양지 쇠고기 쌀국수, 세트(월남쌈+양지or해물쌀국수2개), 반세오, 월남쌈, 카오팟...",053-939-3688,(지번) 대구시 북구 산격동 1416-49
6,평가중,봉봉해물탕,"해물탕\n, 해물찜",053-383-8000,대구 북구 유통단지로7길 21
7,4.3,72번가,"준벅, 카우보이정식, 코스모 폴리탄, 프렌치 키스",053-939-0817,(지번) 대구 북구 산격동 1393-1
8,4.3,META PIZZA,"미트디럭스피자, 쉬림프토파토 파스타, 로제, 프리마베라 피자, 쉬림프크림 파스타",053-939-6767,대구광역시 북구 대학로 103 지하1층
9,평가중,모모야스시 참치,"모듬 스시 11P, 특스시 13P, 모모야스시 11P, 참치메뉴, 광어추가회",053-381-8999,대구광역시 북구 대불로 37


In [29]:
df.to_csv('맛집.tsv', sep='\t', index=False)